In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# Load the data
data = pd.read_csv('Housing_Data.csv')

# Inspect the data
print(data.head())
print(data.info())
print(data.describe())
print(data.columns)

In [ ]:
target_column = 'price'

In [ ]:
# Identify numerical and categorical columns
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns.drop(target_column)
categorical_features = data.select_dtypes(include=['object']).columns

# Preprocessing for numerical data: impute missing values and scale
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define the model
model = RandomForestRegressor()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])



In [ ]:
X = data.drop(target_column, axis=1)
y = data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train the model
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)


In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [ ]:
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
# Optimize Hyperparameters

# Define the parameter grid
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 10, 20, 30]
}

In [ ]:
# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

In [ ]:
# Best parameters
print(grid_search.best_params_)

In [ ]:
# Evaluate the best model
best_model = grid_search.best_estimator_

In [ ]:
# Make predictions with the best model
y_pred_best = best_model.predict(X_test)

In [ ]:
# Evaluate the best model
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)


In [ ]:
print(f'Best Mean Squared Error: {mse_best}')
print(f'Best R-squared: {r2_best}')

In [ ]:
# Documentation
print("""
# Home Value Prediction Model Documentation

## Model Architecture
The model uses a Random Forest Regressor with a preprocessing pipeline that handles missing values and encodes categorical variables.

## Training Process
1. **Data Preprocessing**:
    - Missing values in numerical columns are imputed with the mean.
    - Missing values in categorical columns are imputed with the most frequent value.
    - Categorical columns are one-hot encoded.

2. **Model Training**:
    - The model is trained using the RandomForestRegressor.
    - Hyperparameters are optimized using GridSearchCV.

## Usage Instructions
1. Load the required libraries.
2. Preprocess the data as described.
3. Train the model using the provided pipeline.
4. Evaluate the model performance.
5. Fine-tune the model based on evaluation metrics.
6. Validate predictions on unseen data.

## Model Performance
- **Mean Squared Error**: <insert value>
- **R-squared**: <insert value>
""")